In [1]:
# Extract from https://github.com/StanfordVL/behavior/blob/main/behavior/benchmark/behavior_benchmark.py
import os
import bddl
import json
import behavior
import pyquaternion  
import numpy as np
from PIL import Image
from igibson.utils.utils import parse_config
from igibson.envs.igibson_env import iGibsonEnv

from igibson.custom_utils import get_env_config, render_robot_pov, generate_scenes, generate_scene


 _   _____  _  _
(_) / ____|(_)| |
 _ | |  __  _ | |__   ___   ___   _ __
| || | |_ || || '_ \ / __| / _ \ | '_ \
| || |__| || || |_) |\__ \| (_) || | | |
|_| \_____||_||_.__/ |___/ \___/ |_| |_|



/miniconda/envs/igibson/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
activities = sorted(
        [
            item
            for item in os.listdir(os.path.join(os.path.dirname(bddl.__file__), "activity_definitions"))
            if item != "domain_igibson.bddl"
        ]
    )

In [3]:
assert len(activities)==100, f"Only {len(activities)} activities found."

In [4]:
scene_json = os.path.join(
            os.path.dirname(bddl.__file__),
            "activity_to_preselected_scenes.json",
        )

with open(scene_json) as f:
    activity_to_scenes = json.load(f)

print(len(activity_to_scenes))

143


## Dev

In [5]:
os.path.dirname(behavior.__file__)

'/scratch/cs/world-models/dainesn1/behavior/behavior'

In [6]:
env_config_file = os.path.join(
            os.path.dirname(behavior.__file__),
            "configs/behavior_full_observability.yaml",
        )
env_config = parse_config(env_config_file)
env_config

{'scene': 'igibson',
 'scene_id': 'Beechwood_1_int',
 'clutter': False,
 'build_graph': True,
 'load_texture': True,
 'pybullet_load_texture': True,
 'should_open_all_doors': False,
 'texture_randomization_freq': None,
 'object_randomization_freq': None,
 'robot': {'name': 'BehaviorRobot'},
 'task': 'cleaning_cupboards',
 'task_id': 0,
 'instance_id': 0,
 'online_sampling': False,
 'discount_factor': 0.99,
 'max_step': 500,
 'output': ['proprioception',
  'rgb',
  'highlight',
  'depth',
  'seg',
  'ins_seg',
  'task_obs'],
 'fisheye': False,
 'image_width': 128,
 'image_height': 128,
 'vertical_fov': 120,
 'depth_low': 0.0,
 'depth_high': 10.0,
 'depth_noise_rate': 0.0,
 'scan_noise_rate': 0.0}

In [7]:
# Modify env_config - later we can make our own
env_config['image_width'] = 720
env_config['image_height'] = 720
# No idea what all this stuff is, but we might want to remove everything that is not useful once we inspect it
env_config['output'] = ['proprioception','rgb','highlight','depth','seg','ins_seg','task_obs']
env_config

{'scene': 'igibson',
 'scene_id': 'Beechwood_1_int',
 'clutter': False,
 'build_graph': True,
 'load_texture': True,
 'pybullet_load_texture': True,
 'should_open_all_doors': False,
 'texture_randomization_freq': None,
 'object_randomization_freq': None,
 'robot': {'name': 'BehaviorRobot'},
 'task': 'cleaning_cupboards',
 'task_id': 0,
 'instance_id': 0,
 'online_sampling': False,
 'discount_factor': 0.99,
 'max_step': 500,
 'output': ['proprioception',
  'rgb',
  'highlight',
  'depth',
  'seg',
  'ins_seg',
  'task_obs'],
 'fisheye': False,
 'image_width': 720,
 'image_height': 720,
 'vertical_fov': 120,
 'depth_low': 0.0,
 'depth_high': 10.0,
 'depth_noise_rate': 0.0,
 'scan_noise_rate': 0.0}

## Helper functions

In [8]:
#scene_id =  "Beechwood_0_int"
#task = "assembling_gift_baskets"
#env_config = get_env_config()
#env_config["scene_id"] = scene_id
#env_config["task"] = task
#env_config["task_id"] = 0
#env_config["instance_id"] = 0

# Keep this in memory as it's removed from the config when we init iGibson - so we need to reload it
#robot_name = env_config["robot"]["name"]

#env = iGibsonEnv(
#        config_file=env_config,
#        mode="headless",
#        action_timestep=1.0 / 30.0,
#        physics_timestep=1.0 / 120.0,
#    )

#render_robot_pov(env, env_config, step='initial')

## Full loop

In [9]:
for activity in activities:
    assert activity in activity_to_scenes.keys()
    scenes = sorted(set(activity_to_scenes[activity]))  # Get the scenes where the activity can be performed
    num_scenes = len(scenes)
    assert num_scenes <= 3

    # Official benchmark: Evaluate 9 episodes per activity
    # instance id 0-9: seen poses
    # instance id 10-19: unseen poses
    # instance id 20-29: unseen scenes (same houses but different furniture arrangement)
    # If you don't have enough activity instances in ig_dataset, the behavior data bundle is outdated.
    # You need to follow the participant guide and download again.

    # Depending on the number of scenes available for this activity
    if num_scenes == 3:
        # Regular case. We use three instances per scene (one "seen poses", one "unseen poses" and one
        # "unseen scene" (different furniture))
        scene_instance_ids = {
            scenes[0]: [0, 10, 20],
            scenes[1]: [0, 10, 20],
            scenes[2]: [0, 10, 20],
        }
    elif num_scenes == 2:
        # We use four instances of one scene (two "seen poses", one "unseen poses" and one
        # "unseen scene" (different furniture)) and five from another scene (two "seen poses", two
        # "unseen poses" and one "unseen scene" (different furniture))
        scene_instance_ids = {
            scenes[0]: [0, 1, 10, 20],
            scenes[1]: [0, 1, 10, 11, 20],
        }
    else:
        # We use nine instances of one scene (three "seen poses", three "unseen poses" and three
        # "unseen scene" (different furniture))
        scene_instance_ids = {scenes[0]: [0, 1, 2, 10, 11, 12, 20, 21, 22]}

    # Generate scene and useful info about it
    generate_scenes(activity, scene_instance_ids, gen_only_first=True)


 assembling_gift_baskets Beechwood_0_int 0
Rendering settings is None - going with the default settings!


scene quality check: bottom_cabinet_84 has joint that cannot extend for >66%.
/miniconda/envs/igibson/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
Robot's cached initial pose has collisions.


																																																																																																																																																																																											b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
left_hand_shoulderb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
right_hand_shoulderb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
neckb3Warning[examples/Importers/Impor


KeyboardInterrupt

